In [2]:
from DDDS.drive import Drive
import pandas as pd
import re

In [3]:
drive = Drive()

Connected successfully!


In [87]:
folders = drive.list('folder')
folders

[{'id': '1-AspS9GBqIj-pzA5KM6F03o-9fviuBq2', 'name': 'échantillon'},
 {'id': '1-yIXpLgb_gMzgnaT2Wxl0ckHhRmp47KU', 'name': 'Video validations'},
 {'id': '17UaFCmIylltkftISr0rthJoSS66bJYlt', 'name': 'Validations'},
 {'id': '1i22RM6oJNZr8opAhLK8BrRZTfjh5caOC', 'name': 'Video annotations'},
 {'id': '15nrMZnD_YgMchc43zIFoyWxEMiXYVIP9', 'name': 'data non utilisés'},
 {'id': '15lelEwAZMbPUeUhFrjiwo-hN6d1up8H8',
  'name': 'Videos archives non utilsés'},
 {'id': '1i-BUr1yQTLSXl2dk5US7ZNtrdZHXhQ0n', 'name': 'non utilisé'},
 {'id': '13hPk0MqNH_yNnRGRNTIMUfyT_4HfxJuG', 'name': 'auto evaluation'},
 {'id': '1-IHttqHaNoz0nKEh5TPPGUh7xeKFhBnr', 'name': 'capture ecrans'},
 {'id': '1-2v2QM1S74j789YGMkUInShLyQVu_gjL', 'name': 'annotations'}]

In [ ]:
# folder ID with validated annotation files
validations_folder = 

In [12]:
# Files in "Video validations" folder
validated_videos = drive.list('csv', add_query="'1-yIXpLgb_gMzgnaT2Wxl0ckHhRmp47KU' in parents")

In [13]:
validated_videos

[{'id': '11D-Lanp1cp03piT0y7FcvdTP3b27LP9O',
  'name': 'Christophe-2021-10-28 09-28-56 844.flv.csv'},
 {'id': '11mnlOUXETTsZO6w0R5jCxKkPPDcosNpL',
  'name': 'validateur-2021-11-22 15-40-50 eb0.flv.csv'},
 {'id': '1HEjJJt7lX77D2dO8c9p6hR5KYYtK3FSC',
  'name': 'validateur-2021-11-25 13-45-12 ed2.flv.csv'},
 {'id': '11_6AdcMIdoajeJ3PqToNWxpYmvZszP5o',
  'name': 'Christophe-2021-11-17 10-58-38 e99.flv.csv'},
 {'id': '11YSruFD5BIvon23xF0wu1GYX0mzSGNx9',
  'name': 'Christophe-2021-10-25 10-31-58 c27.flv.csv'},
 {'id': '1FQIYDGcHM4o7Olhgg0DQr5aU2UGxtYGr',
  'name': 'validateur-2021-11-04 13-51-55 ed2.flv.csv'},
 {'id': '18o3ZjAVuQoqkLlYfTd_Skvv7u34GuqSV',
  'name': 'Zakaria-2021-12-01 13-20-00 e99.flv.csv'},
 {'id': '18kM5KV4h3riTCRUxG1lcSuBuGOr5yTgC',
  'name': 'Zakaria-2021-11-24 15-36-04 e99.flv.csv'},
 {'id': '18i0DQAsu9hAV17Cj3FtawH0grIKpO9Bs',
  'name': 'Zakaria-2021-12-07 14-35-11 ea9.flv.csv'},
 {'id': '11YLe6ccK6uf751T0KwG_n-fJx_2OwEU3',
  'name': 'Christophe-2021-11-30 16-16-03 ece.

In [20]:
def convert_video_names(file_name):
    space_split = file_name.split(' ')
    date_split = space_split[0].split('-')
    time_split = space_split[1].split('-')
    driver_id = space_split[-1].split('.')
    # year, month, day, hour, minute, driver_id
    return (date_split[1], date_split[2], date_split[3], time_split[0], time_split[1], driver_id[0])

In [32]:
date = convert_video_names(validated_videos[0]['name'])

In [53]:
def generate_hrv_filename(name):
    return f"{name[2]}_{name[1]}_{name[0]}", name[5]

In [56]:
date, driver = generate_hrv_filename(date)
date

'28_10_2021'

In [63]:
drive.list('txt', add_query=f"(name contains '{date}' and name contains '{driver}')")

[{'id': '11IVOnnmA_Cu_LCRKxvVwc0RFQldw-CGn',
  'name': '28_10_2021_09_22 844.txt'}]

In [86]:
drive.list('csv', add_query=f"(name contains 'annotation_{date}' and name contains '{driver}')")

[{'id': '11KmJHEHeZC1WSx_zRbcW27AaAPxrajtK',
  'name': 'annotation_28_10_2021_09_22 844.csv'}]

In [ ]:
def convert_HRV(file_name):
    ''' all files are converted into the format
    yyyy/mm/dd + 3 digit code if included'''
    
    name = file_name.split(' ')
    
    first = name[0].split('_')
    
    # handles 15_12_2021_11_23_garmin.txt file names
    if first[-1] == 'garmin.txt':
        first[0], first[1], first[2] = first[2], first[1], first[0]
        last = ' '.join(first[0:3])
        
    # handles 05_11_2021_13_34_garmin ea9.txt file names
    elif first[-1] == 'garmin':
        name = name[1].split('.')
        first[0], first[1], first[2], first[3] = first[2], first[1], first[0], name[0]
        last = ' '.join(first[0:4])
        
    # handles 05_11_2021_13_34 ea9.txt file names
    elif len(name) > 1:
        name = name[1].split('.')
        first[0], first[1], first[2], first[3] = first[2], first[1], first[0], name[0]
        last = ' '.join(first[0:4])
        
    # handles 04_11_2021_10_52.txt file names
    else:
        first[0], first[1], first[2] = first[2], first[1], first[0]
        last = ' '.join(first[0:3])
        
    return last

In [11]:
# Create list of Headers for each type of HRV files
headers_list_simple_sensor = ['Timestamp_Google', 'Timestamp_Device', 'Device_id', 'Heart_Rate', 'RR_rate']
headers_list_garmin_sensor = ['Timestamp_Google', 'Timestamp_Device', 'RR_rate']
headers_list_simple_sensor_2 = ['Timestamp_Google', 'Device_id', 'Heart_Rate', 'RR_rate']

In [146]:
hrv = drive.download('15ZzttyiK0qXJFsqtusqDMHi83G9dBv55')

Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [147]:
hrv_df = pd.read_csv(hrv, sep=';', names=headers_list_simple_sensor)

In [148]:
hrv_df

,Timestamp_Google,Timestamp_Device,Device_id,Heart_Rate,RR_rate
0,1637844179265,1637844179588,CD:87:AB:E8:05:12,88,[806]
1,1637844179364,1637844179681,CA:93:07:67:1E:FF,78,"[781, 753]"
2,1637844179406,1637844179728,DC:92:21:A0:6D:B7,79,[]
3,1637844179445,1637844179775,DC:92:21:A0:6D:B7,79,[]
4,1637844179929,1637844180244,DC:92:21:A0:6D:B7,79,[]
...,...,...,...,...,...
25747,1637848455621,1637848455950,DC:92:21:A0:6D:B7,73,[]
25748,1637848455650,1637848455981,DC:92:21:A0:6D:B7,73,[]
25749,1637848456106,1637848456434,DC:92:21:A0:6D:B7,74,[]
25750,1637848456129,1637848456450,DC:92:21:A0:6D:B7,74,[]


In [150]:
def timestamp_formatting(df, *args):
    for timestamp in args:
        if timestamp in df.columns:
            df[timestamp] = df[timestamp].apply(lambda x: pd.Timestamp(x, unit="ms"))
    return df

In [151]:
hrv_df = timestamp_formatting(hrv_df, 'Timestamp_Google', 'Timestamp_Device')

In [152]:
hrv_df

,Timestamp_Google,Timestamp_Device,Device_id,Heart_Rate,RR_rate
0,2021-11-25 12:42:59.265,2021-11-25 12:42:59.588,CD:87:AB:E8:05:12,88,[806]
1,2021-11-25 12:42:59.364,2021-11-25 12:42:59.681,CA:93:07:67:1E:FF,78,"[781, 753]"
2,2021-11-25 12:42:59.406,2021-11-25 12:42:59.728,DC:92:21:A0:6D:B7,79,[]
3,2021-11-25 12:42:59.445,2021-11-25 12:42:59.775,DC:92:21:A0:6D:B7,79,[]
4,2021-11-25 12:42:59.929,2021-11-25 12:43:00.244,DC:92:21:A0:6D:B7,79,[]
...,...,...,...,...,...
25747,2021-11-25 13:54:15.621,2021-11-25 13:54:15.950,DC:92:21:A0:6D:B7,73,[]
25748,2021-11-25 13:54:15.650,2021-11-25 13:54:15.981,DC:92:21:A0:6D:B7,73,[]
25749,2021-11-25 13:54:16.106,2021-11-25 13:54:16.434,DC:92:21:A0:6D:B7,74,[]
25750,2021-11-25 13:54:16.129,2021-11-25 13:54:16.450,DC:92:21:A0:6D:B7,74,[]


In [161]:
exp_start = {'timestamp_google': pd.Timestamp(1637844329071, unit='ms'), 'ts_machine': pd.Timestamp(1637844329389, unit='ms')}

In [162]:
exp_start

{'timestamp_google': Timestamp('2021-11-25 12:45:29.071000'),
 'ts_machine': Timestamp('2021-11-25 12:45:29.389000')}

In [169]:
annotations_df = pd.read_csv(drive.download('1HEjJJt7lX77D2dO8c9p6hR5KYYtK3FSC'), index_col=0).drop(columns='timestamp_lena')

Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [172]:
annotations_df['aligned_instant'] = annotations_df['Instant'] - annotations_df.iloc[0]['Instant']
annotations_df

,Instant,duree,evenement,validation,aligned_instant
0,16388,1.000000,Experience start,1,0
1,372466,1.813908,Debut conduite,0,356078
2,374301,0.341190,Debut conduite,1,357913
3,408132,1.973998,C.Position,0,391744
4,535656,0.934461,Grattez\frottez,1,519268
...,...,...,...,...,...
314,3769101,2.321809,C.Position,1,3752713
315,3888199,7.406835,Baillements,1,3871811
316,3892496,2.618438,Baillements,1,3876108
317,4053177,2.763184,Fin Experience,0,4036789


In [178]:
annotations_df['Timestamp_Google'] = pd.to_timedelta(annotations_df['aligned_instant'], unit='ms') + exp_start['timestamp_google']

In [179]:
annotations_df

,Instant,duree,evenement,validation,aligned_instant,Timestamp_Google
0,16388,1.000000,Experience start,1,0,2021-11-25 12:45:29.071
1,372466,1.813908,Debut conduite,0,356078,2021-11-25 12:51:25.149
2,374301,0.341190,Debut conduite,1,357913,2021-11-25 12:51:26.984
3,408132,1.973998,C.Position,0,391744,2021-11-25 12:52:00.815
4,535656,0.934461,Grattez\frottez,1,519268,2021-11-25 12:54:08.339
...,...,...,...,...,...,...
314,3769101,2.321809,C.Position,1,3752713,2021-11-25 13:48:01.784
315,3888199,7.406835,Baillements,1,3871811,2021-11-25 13:50:00.882
316,3892496,2.618438,Baillements,1,3876108,2021-11-25 13:50:05.179
317,4053177,2.763184,Fin Experience,0,4036789,2021-11-25 13:52:45.860
